# Globus Crawler Skluma Notebook

### Use the Globus Python SDK to create authenticated user and run batch metadata extraction job on its files. 

#### TODO: Refresh tokens, move root-dir to top of file, separate settable vars from other Globus functionality, set up Jetstream instance, send files to Jetstream instance, users configure sending data via REST to THEIR OWN instance, fix 

In [1]:
import globus_sdk

In [6]:
CLIENT_ID = "8ac7dc37-ef09-4c7b-bb24-bca520451842"
ENDPOINT_UUID = "3a261574-3a83-11e8-b997-0ac6873fc732"

# start Native App Grant, and print out the URL where users login as part of the flow (step 2 above)

# create a client object that tracks state as we do this flow
native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
# explicitly start the flow (some clients may support multiple flows)
native_auth_client.oauth2_start_flow()
# print URL
print("Login Here:\n\n{0}".format(native_auth_client.oauth2_get_authorize_url()))

print(("\n\nNote that this link can only be used once! "
       "If login or a later step in the flow fails, you must restart it."))


Login Here:

https://auth.globus.org/v2/oauth2/authorize?client_id=8ac7dc37-ef09-4c7b-bb24-bca520451842&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=K0N-d-oSuBSRKsoe_Jpr7ndwLrbr3LrhRH0vDFZMiCA&code_challenge_method=S256&access_type=online


Note that this link can only be used once! If login or a later step in the flow fails, you must restart it.


In [4]:
auth_code = "RPJ84qLFMDOOkFIY30Lrf3UG0xff3b"
# and exchange it for a response object containing your token(s)
token_response = native_auth_client.oauth2_exchange_code_for_tokens(auth_code)

# extract the Access Token for Globus Transfer, known as "transfer.api.globus.org"
transfer_access_token = token_response.by_resource_server['transfer.api.globus.org']['access_token']

# wrap the token in an object that implements the globus_sdk.GlobusAuthorizer interface
# in this case, an AccessTokenAuthorizer, which takes an access token and produces Bearer Auth headers
transfer_authorizer = globus_sdk.AccessTokenAuthorizer(transfer_access_token)

# create a TransferClient object which Authorizes its calls using that GlobusAuthorizer
tc = globus_sdk.TransferClient(authorizer=transfer_authorizer)

In [8]:
try: 
    r = tc.get_endpoint(ENDPOINT_UUID)
    print("HTTP Status Code:", r.http_status)
    print("Content Type:", r.content_type)
    print("Data:", r)
    print("Endpoint Display Name:", r["display_name"]) # shortcut for r.data["display_name"]
    
except globus_sdk.GlobusAPIError as ex:
    print("HTTP Status Code:", ex.http_status)
    print("Error Code      :", ex.code)
    print("Error Message   :", ex.message)

HTTP Status Code: 200
Content Type: application/json
Data: {
  "DATA": [
    {
      "DATA_TYPE": "server",
      "hostname": "petreldtn7.pub.jlse.anl.gov",
      "id": 333321,
      "incoming_data_port_end": null,
      "incoming_data_port_start": null,
      "is_connected": true,
      "is_paused": false,
      "outgoing_data_port_end": null,
      "outgoing_data_port_start": null,
      "port": 2811,
      "scheme": "gsiftp",
      "subject": "/C=US/O=Globus Consortium/OU=Globus Connect Service/CN=89a817c8-b504-11e7-b0b9-22000a92523b",
      "uri": "gsiftp://petreldtn7.pub.jlse.anl.gov:2811"
    },
    {
      "DATA_TYPE": "server",
      "hostname": "petreldtn8.pub.jlse.anl.gov",
      "id": 333445,
      "incoming_data_port_end": null,
      "incoming_data_port_start": null,
      "is_connected": true,
      "is_paused": false,
      "outgoing_data_port_end": null,
      "outgoing_data_port_start": null,
      "port": 2811,
      "scheme": "gsiftp",
      "subject": "/C=US/O=Globu

In [ ]:
### Create a DFS by iteratively searching for directories in directories. 
endpoint_id = ENDPOINT_UUID
endpoint_path = "/"

visited = set()
unvisited_list = [endpoint_path]

while unvisited_list: # while unvisited elements.
    
    current_path = unvisited_list[-1]
    unvisited_list.pop()
        
    visited.add(current_path)
    r = tc.operation_ls(endpoint_id, path=current_path)
    for item in r:
        if item["type"] == 'dir' and current_path + "/" + item["name"] + "/" not in visited:
            unvisited_list.append(current_path + "/" + item["name"] + "/")
            print(item["name"])